In [1]:
!pip -q install arxiv langchain-groq faiss-cpu langchain-openai wikipedia langchain_huggingface langchain_community langchain


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepspeed 0.10.3 requires pydantic<2.0.0, but you have pydantic 2.10.6 which is incompatible.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.26.1 which is incompatible.


In [2]:
!pip -q install langchain[all]

In [3]:
#tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)

In [5]:
wikipedia_query_run=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [6]:
wikipedia_query_run

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [7]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_query_run=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [8]:
tools=[wikipedia_query_run,arxiv_query_run]

In [20]:
## create custom tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

In [16]:
loader=WebBaseLoader("https://www.langchain.com/langsmith")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about langsmith")
retriever_tool.name

'langsmith-search'

In [18]:
tools=[wikipedia_query_run,arxiv_query_run,retriever_tool]

In [43]:
#run all the tools with agents and llm model
#LLM Agent Executor
llm=ChatGroq(model_name="llama-3.1-8b-instant")

In [44]:
#Prompt Template Hub
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")

In [45]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [50]:
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
agent=create_openai_tools_agent(llm,tools,prompt)
agent_executor=AgentExecutor(agent=agent, tools=tools,verbose=True)

In [51]:
#agent_executor

In [52]:
agent_executor.invoke({"input":"what is the name of the prime minister of india?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'current prime minister of india'}`


Page: Prime Minister of India
Summary: The prime minister of India (ISO: Bhārata kē Pradhānamantrī) is the head of government of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers, despite t
Invoking: `wikipedia` with `{'query': 'current prime minister of india 2024'}`


Page: Prime Minister of India
Summary: The prime minister of India (ISO: Bhārata kē Pradhānamantrī) is the head of government of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers, despite tThe current Prime Minister of India is Narendra Modi.

> Finished chain.


{'input': 'what is the name of the prime minister of india?',
 'output': 'The current Prime Minister of India is Narendra Modi.'}

In [ ]:
agent_executor.invoke({"input":"what is the price of langsmith? is it free?"})

In [ ]:
agent_executor.invoke({"input":"what is the paper "})